In [27]:
import pandas as pd

### Combien de joueurs par plateforme? 
### (en utilisant la table "players" qui contient tous les joueurs inscrits)

In [28]:
players=pd.read_csv("final/players.csv",index_col=0)

In [29]:
players.head()

,playerid,country,plateforme,created,created_year
0,2744567_playstation,Croatia,playstation,unknown,NaN
1,328452_playstation,United States,playstation,unknown,NaN
2,2521482_playstation,United Kingdom,playstation,unknown,NaN
3,390175_playstation,"Korea, Republic of",playstation,unknown,NaN
4,4348945_playstation,Russian Federation,playstation,unknown,NaN


In [30]:
# Requete finale pour répondre a la quesiton : Combien de joueurs par plateforme ?

registered_players_per_platform = players.groupby('plateforme')['playerid'].nunique().reset_index()
registered_players_per_platform

,plateforme,playerid
0,playstation,356600
1,steam,424683
2,xbox,274450


In [31]:
registered_players_per_platform.columns=['plateforme','registered_players']
registered_players_per_platform

,plateforme,registered_players
0,playstation,356600
1,steam,424683
2,xbox,274450


### Combien de joueurs par plateforme? (en utilisant la table "purchased_games" qui contient les joueurs qui ont acheté)

In [32]:
# importer la table crée dans la réponse à la question : Top 10 jeux pas pays (fichier top_10_jeux_par_pays.ipynb). 

purchased_games_split=pd.read_csv("powerbi/purchased_games_splitlibrary.csv",index_col=0)

In [33]:
purchased_games_split.head()

,playerid,gameid,plateforme
0,268071_playstation,14972_playstation,playstation
0,268071_playstation,417905_playstation,playstation
0,268071_playstation,14693_playstation,playstation
0,268071_playstation,7742_playstation,playstation
0,268071_playstation,20162_playstation,playstation


In [34]:
# Requete finale pour répondre a la quesiton : Combien de joueurs par plateforme ?

purchasing_players_per_platform = purchased_games_split.groupby('plateforme')['playerid'].nunique().reset_index().sort_values('playerid', ascending=False)

purchasing_players_per_platform

,plateforme,playerid
1,steam,102548
0,playstation,46582
2,xbox,46466


In [35]:
# renommer la colonne
purchasing_players_per_platform.columns=['plateforme','purchasing_players']
purchasing_players_per_platform

,plateforme,purchasing_players
1,steam,102548
0,playstation,46582
2,xbox,46466


In [36]:
# merger deux tables pour les info completes pour chaque plateforme

players_per_platform=registered_players_per_platform.merge(
  purchasing_players_per_platform,
  left_on =['plateforme'],
  right_on =['plateforme'],
  how='left'    
)

players_per_platform

,plateforme,registered_players,purchasing_players
0,playstation,356600,46582
1,steam,424683,102548
2,xbox,274450,46466


In [37]:
# poucentage des joueurs actifs par plateforme 

players_per_platform['conversion_rate']=players_per_platform['purchasing_players']/players_per_platform['registered_players']
players_per_platform


,plateforme,registered_players,purchasing_players,conversion_rate
0,playstation,356600,46582,0.130628
1,steam,424683,102548,0.241470
2,xbox,274450,46466,0.169306


### Analyse complémentaire sur la table purchased_games

nombre de joueurs par jeu

In [39]:
players_per_game = purchased_games_split.groupby('gameid')['playerid'].nunique().reset_index().sort_values('playerid', ascending=False)

In [40]:
players_per_game.columns = ["gameid", "number_of_players"]

In [41]:
players_per_game

,gameid,number_of_players
93030,unknown_steam,55607
72628,730_steam,41807
75756,7779_playstation,29802
80449,9459_playstation,28900
6712,137118_playstation,28587
...,...,...
87218,391740_steam,1
87217,391731_xbox,1
66124,667831_playstation,1
87214,391619_playstation,1


In [51]:
# mediane de nombre joueur par jeu 
players_per_game.describe()

,number_of_players
count,93031.000000
mean,374.805850
std,1337.697549
min,1.000000
25%,9.000000
50%,49.000000
75%,205.000000
max,55607.000000


nombre de jeux par joueur

In [43]:
games_per_player = purchased_games_split.groupby("playerid")["gameid"].nunique().reset_index().sort_values('gameid', ascending=False)
games_per_player.columns = ["playerid", "number_of_games"]

In [44]:
games_per_player

,playerid,number_of_games
110314,76561198094227663_steam,32463
80878,76561197962473290_steam,28196
83433,76561197976968076_steam,25707
93734,76561198027214426_steam,25100
162067,76561198842864763_steam,25085
...,...,...
150412,76561198359116498_steam,1
112263,76561198103910377_steam,1
112261,76561198103893957_steam,1
150415,76561198359154280_steam,1


In [ ]:
# mediane de jeu par joueur
games_per_player.describe()

,number_of_games
count,195596.000000
mean,178.268283
std,502.481386
min,1.000000
25%,1.000000
50%,71.000000
75%,202.000000
max,32463.000000


nombre de jeux par plateforme

In [46]:
games_per_platform = purchased_games_split.groupby('plateforme')['gameid'].nunique().reset_index().sort_values('gameid', ascending=False)


In [47]:
games_per_platform.columns = ["plateforme", "number_of_games"]

In [48]:
games_per_platform

,plateforme,number_of_games
1,steam,45093
0,playstation,33468
2,xbox,14470


In [49]:
# merge pour les info completes les trois plateformes 

players_per_platform=players_per_platform.merge(
  games_per_platform,
  left_on =['plateforme'],
  right_on =['plateforme'],
  how='left'    
)

players_per_platform

,plateforme,registered_players,purchasing_players,conversion_rate,number_of_games
0,playstation,356600,46582,0.130628,33468
1,steam,424683,102548,0.241470,45093
2,xbox,274450,46466,0.169306,14470


In [50]:
# enregistrer en csv 

players_per_platform.to_csv("powerbi/nb_joueurs_par_plateforme.csv", index=False)